## Simple Match History Test
-----------------------------

This notebook uses [Pantheon](https://github.com/Canisback/pantheon) to define various other functions for using the Riot API. The reason for using Pantheon is to make use of it's rate limiting system, since standalone developers can only make so many calls to the API without getting blacklisted.

In [2]:
import json, asyncio, nest_asyncio, datetime, requests
import pandas as pd #data visualization
from pantheon import pantheon
nest_asyncio.apply()

Insert your own API key into the **api_key** variable in the following cell if you intend to run this.

In [3]:
server = "na1"
api_key = "YOUR API KEY GOES HERE"

These are some functions provided in [Pantheon](https://github.com/Canisback/pantheon) that fetch any account's summoner ID and recent match history.

In [4]:
def requestsLog(url, status, headers):
    print(url)
    print(status)
    print(headers)

panth = pantheon.Pantheon(server, api_key, errorHandling=True, requestsLoggingFunction=requestsLog, debug=True)

async def getSummonerId(name):
    try:
        data = await panth.getSummonerByName(name)
        return (data['id'],data['accountId'])
    except Exception as e:
        print(e)


async def getRecentMatchlist(accountId):
    try:
        data = await panth.getMatchlist(accountId, params={"endIndex":20})
        return data
    except Exception as e:
        print(e)

async def getRecentMatches(accountId):
    try:
        matchlist = await getRecentMatchlist(accountId)
        tasks = [panth.getMatch(match['gameId']) for match in matchlist['matches']]
        return await asyncio.gather(*tasks)
    except Exception as e:
        print(e)
        

Change the **name** variable to search for different summoners. Feel free to try with your own summoner name!

In [5]:
name = "Jooku"

#call function to set the summonerId and accountId for the name that I chose
loop = asyncio.get_event_loop()  
(summonerId, accountId) = loop.run_until_complete(getSummonerId(name))


https://na1.api.riotgames.com/lol/summoner/v4/summoners/by-name/Jooku
200
<CIMultiDictProxy('Date': 'Thu, 16 Sep 2021 00:18:37 GMT', 'Content-Type': 'application/json;charset=utf-8', 'Transfer-Encoding': 'chunked', 'Connection': 'keep-alive', 'Vary': 'Accept-Encoding', 'X-App-Rate-Limit': '20:1,100:120', 'X-App-Rate-Limit-Count': '1:1,1:120', 'X-Method-Rate-Limit': '2000:60', 'X-Method-Rate-Limit-Count': '1:60', 'X-Riot-Edge-Trace-Id': '5e8e567f-3460-4270-919e-75a66e5c258b', 'Content-Encoding': 'gzip', 'Access-Control-Allow-Origin': '*', 'Access-Control-Allow-Methods': 'GET, PUT, DELETE, POST, OPTIONS', 'Access-Control-Allow-Headers': 'DNT,User-Agent,X-Requested-With,If-Modified-Since,Cache-Control,Content-Type,Range', 'Access-Control-Expose-Headers': 'Content-Length,Content-Range')>


Use the **getRecentMatches** function provided by **Pantheon** to get recent match data in JSON format and insert it into a list called **gamesDataRaw**.

In [6]:
%%capture ##this just hides the huge output we would get from the loop
gamesDataRaw = []
gamesDataRaw = loop.run_until_complete(getRecentMatches(accountId))

Now that game data is acquired, define various functions to get various types of game-specific info.

This function will return a summoner's participant ID from a game, given the game data and account ID.

In [7]:
def getParticipantIdFromAccountId(game,accountId):
    for participant in game['participantIdentities']:
        if participant['player']['currentAccountId'] == accountId:
            return participant['participantId']

Define a function to get a participant's team ID (red/blue side) as well as the champion they are playing. 

Note that blue side has team ID 100 and red side has team ID 200.

In [8]:
def getTeamAndChampion(game,participantId):
    for participant in game['participants']:
        if participant['participantId'] == participantId:
            return participant['teamId'],participant['championId'],participant['stats']['win']

This function returns the team compositions for each team in a list format.

In [9]:
def getTeamComposition(game):
   #create an array for each team
    comps = {100:[],200:[]}
    for participant in game['participants']:
        comps[participant['teamId']].append(participant['championId'])
    return comps

Get the patch that the game was played on.

In [10]:
def getPatch(game):
    return ".".join(game['gameVersion'].split(".")[:2])

Get the date that the game was played on.

In [11]:
def getDate(game):
    return datetime.datetime.fromtimestamp(int(game["gameCreation"]/1000)).strftime("%m/%d/%Y, %I:%M")

Get the kills/deaths/assists of a given participant in a game.

In [12]:
def getScore(game,participantId):
    for participant in game['participants']:
        if participant['participantId'] == participantId:
            score = [str(participant['stats']['kills']),str(participant['stats']['deaths']),str(participant['stats']['assists'])]
    return "/".join(score)

Now, call the API to get info for champions and items.

In [13]:
responseChamps = requests.get("http://ddragon.leagueoflegends.com/cdn/11.17.1/data/en_US/champion.json")
championRawData = json.loads(responseChamps.text)

responseItem = requests.get("http://ddragon.leagueoflegends.com/cdn/11.17.1/data/en_US/item.json")
itemRawData = json.loads(responseItem.text)


Create an array called **championIdToName** where champion names are matched with their unique IDs.

In [14]:
championIdToName = {}
for key,champion in championRawData['data'].items():
    championIdToName[int(champion['key'])] = champion['name']

Create an array called **itemIdToName** where item names are matched with their unique IDs.

In [15]:
itemIdToName = {}

for key,item in itemRawData['data'].items():
    itemIdToName[int(key)] = item['name']

Define a function that returns the list of items a participant has in their inventory at the end of a match.

In [16]:
def getItemList(game,participantId):
    for participant in game['participants']:
        if participant['participantId'] == participantId:
            itemsList = []
            for i in range(0,7):
                if not participant['stats']['item'+str(i)] == 0:
                    itemsList.append(itemIdToName[participant['stats']['item'+str(i)]])
            return " / ".join(itemsList)

Use all of the previously defined functions to get data to add to a dataframe.

In [17]:
participantId = getParticipantIdFromAccountId(gamesDataRaw[0],accountId)    

teamId,championId,win = getTeamAndChampion(gamesDataRaw[0],participantId)

compositions = getTeamComposition(gamesDataRaw[0])

playerChampion = championIdToName[championId]

teamComposition = [championIdToName[chId] for chId in compositions[teamId]]
enemyTeamComposition = [championIdToName[chId] for chId in compositions[100 if teamId == 200 else 200]]
"/".join(teamComposition)
"/".join(enemyTeamComposition)



'Alistar/Brand/Viego/Lux/Rumble'

Create a list called **gameInformationList** and fill it with info using the previously defined functions.

In [18]:
gameInformationList = []

for game in gamesDataRaw:
    #get the info that you want
    participantId = getParticipantIdFromAccountId(game,accountId)
    teamId, championId, win = getTeamAndChampion(game,participantId)
    compositions = getTeamComposition(game)
    
    
    #select the info that you want to show
    gameInfo = {}
    gameInfo['win'] = win
    #gameInfo['visionScore'] = visionScore
    gameInfo['champion'] = championIdToName[championId]
    gameInfo['score'] = getScore(game,participantId)
    #work on items, too long
    #gameInfo['items'] = getItemList(game,participantId)
    gameInfo['patch'] = getPatch(game)
    gameInfo['date'] = getDate(game)
    gameInfo['teamComposition'] = "/".join([championIdToName[chId] for chId in compositions[teamId]])
    gameInfo['enemyTeamComposition'] = "/".join([championIdToName[chId] for chId in compositions[100 if teamId == 200 else 200]])
    #add the row to the list
    gameInformationList.append(gameInfo)

Create a dataframe to display **gameInformationList** in an easily readable format.

In [19]:
dataframe = pd.DataFrame(gameInformationList)
dataframe.head(51)

,win,champion,score,patch,date,teamComposition,enemyTeamComposition
0,False,Soraka,0/11/27,11.18,"09/14/2021, 09:17",Caitlyn/Akshan/Soraka/Lucian/Mordekaiser,Alistar/Brand/Viego/Lux/Rumble
1,False,Evelynn,10/9/20,11.18,"09/14/2021, 08:54",Lee Sin/Evelynn/Tahm Kench/Janna/Tristana,Alistar/Cassiopeia/Darius/Master Yi/Kai'Sa
2,True,Twitch,20/0/8,11.18,"09/11/2021, 10:15",Caitlyn/Mordekaiser/Poppy/Yuumi/Twitch,Tristana/Udyr/Katarina/Soraka/Galio
3,True,LeBlanc,15/2/9,11.18,"09/11/2021, 09:53",Yuumi/LeBlanc/Darius/Ziggs/Samira,Sivir/Nocturne/Katarina/Zilean/Leona
4,True,Kalista,30/2/0,11.18,"09/10/2021, 11:03",Caitlyn/Miss Fortune/Kalista/Soraka/Viego,Ashe/Trundle/Annie/Taric/Cho'Gath
5,True,Ezreal,6/1/3,11.17,"09/08/2021, 05:21",Ezreal/Brand/Corki/Graves/Gwen,Renekton/Zed/Karma/Lee Sin/Samira
6,False,Tristana,6/4/12,11.17,"09/08/2021, 04:40",Tristana/Master Yi/Pantheon/Nasus/Galio,Ezreal/Leona/Rengar/Annie/Ornn
7,True,Xayah,11/0/3,11.17,"09/08/2021, 03:38",Xayah/Camille/Pantheon/Lee Sin/Kassadin,Ezreal/Zyra/Graves/Shen/Ahri
8,True,Jhin,20/2/7,11.17,"09/08/2021, 03:00",Ekko/Jhin/Ornn/Lux/Yasuo,Jarvan IV/Teemo/Zed/Heimerdinger/Trundle
9,True,Jhin,5/1/8,11.17,"09/05/2021, 06:43",Zyra/Jhin/Diana/Graves/Sett,Aatrox/Xin Zhao/Yuumi/Ezreal/Vladimir
